In [93]:
import requests
from lxml import etree

article_list = []
for page in range(1,6):
    print(f'第{page}页')
    url = f'https://www.medindia.net/medicalwebsite/indian_healthcategory.asp?alpha=&page={page}&categoryid=9&subcategoryid=42'
    res = requests.get(url)
    html = etree.HTML(res.text)

    articles = html.xpath('//div[@class="report-content"]/article')
    for article in articles:
        name = article.xpath('./div/h4/text()')[0].strip().replace('\r', '')
        url  = article.xpath('./div/span/text()')[0].strip()
        print(url)
        if 'http' not in url:
            continue
        if name == '':
            name = 'Null'
        article_list.append({'NAME':name, 'URL':url})
article_list

第1页
200
http://www.tomarorthopaedics.com
http://amarjyothiclinic.org/index.html
http://dentalclinicdelhi.com

http://www.vitalradiology.com
http://www.pune-dentist.com
http://www.rescuehealthcare.in/
http://www.sankaraeye.com/
http://www.ratandeep.com
http://boneandjointcare.co.in/
第2页
200
http://www.andrology.com/sponsor.htm
http://www.apollohospitals.com
http://www.vellorecmc.org
http://www.choithram.org
http://www.sitalakshmi.com/hospital
http://www.dragarwal.com
http://www.jillahospital.com
http://www.kanti.com
http://www.kghospital.org
http://www.kotharimedical.com
第3页
200
http://members.tripod.com/
http://www.miothospitals.com/
http://www.mvdsc.org
http://www.dynacs.com/
http://www.sgpgi.ac.in
http://www.sksh.org
http://www.speechfoundation.com
http://www.smfhospital.org
http://www.tatamemorialcentre.com
http://www.vardaan.net
第4页
200
http://www.sahajdental.com


IndexError: list index out of range